In [25]:
import os

In [26]:
%pwd

'D:\\Pranshu\\Chicken_Disease_Classification'

In [27]:
os.chdir("D:/Pranshu/Chicken_Disease_Classification")

In [28]:
%pwd

'D:\\Pranshu\\Chicken_Disease_Classification'

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    training_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int 
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float
    trained_model_path: Path
    
@dataclass(frozen=True)
class PreparecallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [30]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PreparecallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PreparecallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories(
            [
                Path(training.root_dir)
            ]
        )

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            training_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            trained_model_path = Path(training.trained_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.learning_rate
        )
        
        return training_config

In [32]:
import time

In [33]:
class PrepareCallback:
    def __init__(self, config: PreparecallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)

    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

In [34]:
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.config.run_functions_eagerly(True)                   # Enable Eager Execution

In [35]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # Recompile the model with fresh optimizer
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
        loss='binary_crossentropy',     # you can use categorical_crossentropy as well
        metrics=['accuracy']
    )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [36]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-05-25 15:46:39,908: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-25 15:46:39,909: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-25 15:46:39,910: INFO: common: created directory at: artifacts]
[2025-05-25 15:46:39,910: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-25 15:46:39,911: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-05-25 15:46:39,912: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


d:\Pranshu\Chicken_Disease_Classification\chicken_disease\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.6691 - loss: 2.2358 - val_accuracy: 0.9375 - val_loss: 0.8600
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 190ms/step - accuracy: 0.9375 - loss: 0.0751 - val_accuracy: 0.9062 - val_loss: 0.9564
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.9250 - loss: 0.8474 - val_accuracy: 0.9062 - val_loss: 1.0427
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 192ms/step - accuracy: 0.9375 - loss: 0.4209 - val_accuracy: 0.9062 - val_loss: 0.9984
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.9460 - loss: 0.3488 - val_accuracy: 0.9531 - val_loss: 0.8440
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 222ms/step - accuracy: 0.9375 - loss: 1.3335 - val_accuracy: 0.9375 - val_loss: 0.7993
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.9609 - loss: 0.2989 - val_accuracy: 0.9375 - val_loss: 0.6019
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - accuracy: 1.0000 - loss: 0.1339 - val_accuracy: 0.9219 - v